# Spatial Modulation , A deep learning based approach for maximising channel capacity for a Massive MIMO system for 5G communication application.

In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import pandas as pd 
import numpy as np
from torch.utils.data.dataset import Dataset




In [214]:
# dataset 
dataset_dir='/content/complete_dataset.csv'

class CustomDataset(Dataset):
  def __init__(self,csv_path):
        self.to_tensor = transforms.ToTensor()
        self.col_Names=["Bits Number","no of antennas","labels"]
        df=pd.read_csv(csv_path,names=self.col_Names)
        self.labels = np.asarray(df.iloc[:,0])
        self.inputdata =np.asarray(df.iloc[:,1])
        self.inputdata1 =np.asarray(df.iloc[:,2])
        self.data_len=len(df.index)

  def __len__(self):
        return self.data_len

  def __getitem__(self, index):
        data=self.inputdata[index]
        data1=self.inputdata1[index]
        data=np.array([data,data1])
        label =np.array(self.labels[index])
        return torch.from_numpy(data),torch.from_numpy(label)


dataset=CustomDataset(dataset_dir)
print(dataset.__getitem__(9)[0].size())

#print(dataframe.head())new_a = a[np.newaxis, :]
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset,[train_size, test_size])

#print(train_dataset[0])



torch.Size([2])


In [215]:
# Hyper-parameters 
input_size = 2
output_size=1
hidden_size = 20
num_classes = 1
num_epochs = 1500
batch_size = 50
learning_rate = 0.005

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)



# Fully connected neural network with one hidden layer
class Spatialmodulation(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Spatialmodulation, self).__init__()
        self.fc1 = nn.Linear(input_size,hidden_size) 
        self.bc1 = nn.BatchNorm1d(hidden_size)
        #self.drp1= nn.Dropout(0.5)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size,hidden_size)  
        self.bc2 = nn.BatchNorm1d(hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size,output_size)  
        self.bc3 = nn.BatchNorm1d(output_size)
       # self.drp2= nn.Dropout(0.5)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(output_size,num_classes) 
        self.relu4 = nn.ReLU()
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fc4(out)
        out = self.relu4(out)
        return out

model = Spatialmodulation(input_size, hidden_size, num_classes)

print(model)

Spatialmodulation(
  (fc1): Linear(in_features=2, out_features=20, bias=True)
  (bc1): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (fc2): Linear(in_features=20, out_features=20, bias=True)
  (bc2): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=20, out_features=1, bias=True)
  (bc3): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc4): Linear(in_features=1, out_features=1, bias=True)
  (relu4): ReLU()
)


In [0]:
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)  

In [0]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (data,labels) in enumerate(train_loader):        
        # Forward pass
        labels=labels[:,np.newaxis]
        outputs = model(data.float())
        loss = criterion(outputs, labels.float())
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % batch_size == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for (data,labels) in test_loader:
        #data=transforms.ToTensor(data)
        outputs = model(data.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/1500], Step [50/148], Loss: 26282.2832
Epoch [1/1500], Step [100/148], Loss: 27574.8418
Epoch [2/1500], Step [50/148], Loss: 18369.5898
Epoch [2/1500], Step [100/148], Loss: 27492.1484
Epoch [3/1500], Step [50/148], Loss: 16613.9043
Epoch [3/1500], Step [100/148], Loss: 13530.2471
Epoch [4/1500], Step [50/148], Loss: 25170.1992
Epoch [4/1500], Step [100/148], Loss: 20311.0195
Epoch [5/1500], Step [50/148], Loss: 15614.0820
Epoch [5/1500], Step [100/148], Loss: 15269.9609
Epoch [6/1500], Step [50/148], Loss: 16524.2129
Epoch [6/1500], Step [100/148], Loss: 13104.1221
Epoch [7/1500], Step [50/148], Loss: 11398.3086
Epoch [7/1500], Step [100/148], Loss: 9123.8477
Epoch [8/1500], Step [50/148], Loss: 8586.7314
Epoch [8/1500], Step [100/148], Loss: 16112.5146
Epoch [9/1500], Step [50/148], Loss: 10316.3389
Epoch [9/1500], Step [100/148], Loss: 11881.9365
Epoch [10/1500], Step [50/148], Loss: 8674.8086
Epoch [10/1500], Step [100/148], Loss: 9945.2754
Epoch [11/1500], Step [50/148], 